In [2]:
pip install ndlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110 kB 8.3 MB/s 
     |████████████████████████████████| 3.1 MB 49.2 MB/s 
  Created wheel for python-igraph: filename=python_igraph-0.9.11-py3-none-any.whl size=9074 sha256=f9e0040218a62cbd86844c6e78b59ab1eeaf8c1d84735ba869925b4b98da4bdb
  Stored in directory: /root/.cache/pip/wheels/7d/e4/0c/0c83f70bd0b99ce8aea47a21f8e52502169e7dd17808d12f30
Successfully built python-igraph


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pickle
import math
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import pandas as pd


In [6]:
G1 = nx.read_edgelist("football.txt", nodetype = int)
data = pd.read_csv("TOPSIS_result_football.csv")
x=data[["DC","BC","CC","EC"]]

In [7]:
x

,DC,BC,CC,EC
0,0.105263,0.032490,0.423792,0.106503
1,0.096491,0.033533,0.422222,0.100915
2,0.096491,0.029161,0.435115,0.090415
3,0.087719,0.028823,0.436782,0.079146
4,0.096491,0.025187,0.404255,0.080489
...,...,...,...,...
110,0.087719,0.006498,0.377483,0.070452
111,0.087719,0.005317,0.391753,0.080728
112,0.061404,0.007790,0.370130,0.048312
113,0.087719,0.004372,0.386441,0.081921


In [8]:
data.drop(columns=['TOPSIS'], inplace=True)
data
AA=pd.read_csv('TOPSIS_result_football.csv')
inf10=AA.head(10)
inf10

,node,DC,CC,BC,EC,TOPSIS
0,0,0.105263,0.423792,0.032490,0.106503,0.924560
1,64,0.096491,0.422222,0.033533,0.100915,0.912863
2,106,0.096491,0.435115,0.029161,0.090415,0.819665
3,41,0.087719,0.436782,0.028823,0.079146,0.786272
4,74,0.096491,0.404255,0.025187,0.080489,0.699216
5,46,0.096491,0.422222,0.024139,0.089273,0.681884
6,80,0.096491,0.423792,0.023836,0.089921,0.673942
7,24,0.105263,0.420664,0.023070,0.106250,0.668289
8,83,0.096491,0.402827,0.023046,0.081215,0.640113
9,4,0.096491,0.423792,0.022213,0.103791,0.638511


In [9]:
Y=inf10[["DC","BC","CC","EC"]]
Yrr=Y.to_numpy()
Yrr

array([[0.10526316, 0.03248995, 0.42379182, 0.10650323],
       [0.09649123, 0.03353296, 0.42222222, 0.10091535],
       [0.09649123, 0.02916105, 0.4351145 , 0.0904148 ],
       [0.0877193 , 0.02882282, 0.43678161, 0.07914559],
       [0.09649123, 0.0251868 , 0.40425532, 0.08048861],
       [0.09649123, 0.02413942, 0.42222222, 0.08927305],
       [0.09649123, 0.02383635, 0.42379182, 0.08992131],
       [0.10526316, 0.0230701 , 0.42066421, 0.10624954],
       [0.09649123, 0.02304632, 0.40282686, 0.08121511],
       [0.09649123, 0.02221345, 0.42379182, 0.10379099]])

In [10]:
kmeans = KMeans(n_clusters=10, init=Yrr, max_iter=20000, n_init=10, random_state=0).fit(x) # l'appel de la fonction k_means 
centr = kmeans.cluster_centers_
centr

/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


array([[0.10526316, 0.01920727, 0.42262564, 0.11734549],
       [0.09899749, 0.02420873, 0.42224043, 0.10499034],
       [0.09649123, 0.02571228, 0.42704285, 0.08986972],
       [0.08479532, 0.02260309, 0.42656369, 0.07382813],
       [0.07346491, 0.01143054, 0.39458047, 0.06326926],
       [0.09244265, 0.0114171 , 0.3956596 , 0.08423813],
       [0.09590643, 0.01743837, 0.40834809, 0.08689012],
       [0.09649123, 0.00684246, 0.37454359, 0.10704751],
       [0.09046053, 0.00751692, 0.38108819, 0.07754673],
       [0.09763539, 0.01272583, 0.40238022, 0.11028363]])

In [11]:
def distance_euclidean(data,center,j): # fonction pour calculer  la distance euclidean 
    T= pd.DataFrame()
    for i in range(len(data)):
        distance=math.sqrt((data['DC'][i]-center[j][0])**2+(data['BC'][i]-center[j][1])**2+
                           (data['CC'][i]-center[j][2])**2+(data['EC'][i]-center[j][3])**2)
        T=T.append({'node':data['node'][i], 'cluster':str(j),'distance from center':distance},ignore_index=True)
    return T.sort_values(by='distance from center')

In [12]:
Res= pd.DataFrame()
for i in range(10):
  T=distance_euclidean(data,centr,i)
  Res=Res.append({'cluster':str(i), 'Node_avant_KMeans':str(inf10['node'][i]),
                    'Node_ apres _KMeans':int(T.head(1)['node']),
                    'distance from center':float(T.head(1)['distance from center'])},ignore_index=True)

In [13]:
Res

,cluster,Node_avant_KMeans,Node_ apres _KMeans,distance from center
0,0,0,28.0,0.002301
1,1,64,4.0,0.003756
2,2,106,80.0,0.003754
3,3,41,67.0,0.007873
4,4,74,49.0,0.005705
5,5,46,9.0,0.005111
6,6,80,94.0,0.003046
7,7,24,59.0,0.007550
8,8,83,104.0,0.003952
9,9,4,34.0,0.004087


In [14]:
Res["Node_ apres _KMeans"]=Res["Node_ apres _KMeans"].astype(int)
Res.to_csv("k_means_result.csv",index=False)

In [15]:
Res

,cluster,Node_avant_KMeans,Node_ apres _KMeans,distance from center
0,0,0,28,0.002301
1,1,64,4,0.003756
2,2,106,80,0.003754
3,3,41,67,0.007873
4,4,74,49,0.005705
5,5,46,9,0.005111
6,6,80,94,0.003046
7,7,24,59,0.007550
8,8,83,104,0.003952
9,9,4,34,0.004087


In [22]:
df=pd.read_csv('TOPSIS_result_football.csv')
df['RankingEC'] = df['EC'].rank(ascending=False)                       # le classement  des resultats 
df['RankingBC'] = df['BC'].rank(ascending=False)
df['RankingCC'] = df['CC'].rank(ascending=False)
top10=pd.DataFrame()
top10['BC']=df[df['RankingBC'].isin(range(1,11))].node.values
top10['CC']=df[df['RankingCC'].isin(range(1,11))].node.values   
top10['EC']=df[df['RankingEC'].isin(range(1,11))].node.values

top10['TOPSIS']=df[df.index.isin(range(10))].node.values
top10.to_csv("top10_result_football.csv")
top10

,BC,CC,EC,TOPSIS
0,0,0,28,0
1,64,106,114,64
2,106,41,103,106
3,41,80,52,41
4,74,4,82,74
5,46,28,101,46
6,80,114,81,80
7,24,35,108,24
8,83,25,61,83
9,4,67,113,4


In [23]:
top10["k_means"]=Res["Node_ apres _KMeans"]

In [24]:
k_top10=top10

In [25]:
k_top10

,BC,CC,EC,TOPSIS,k_means
0,0,0,28,0,28
1,64,106,114,64,4
2,106,41,103,106,80
3,41,80,52,41,67
4,74,4,82,74,49
5,46,28,101,46,9
6,80,114,81,80,94
7,24,35,108,24,59
8,83,25,61,83,104
9,4,67,113,4,34


In [26]:
k_top10.to_csv("k_top10_result_football.csv")